# Reconstruction of integrated wave parameters from the 2D spectrum

Download ERA5 data (e.g. from https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=download)
- all fields have the following dimensions: time, lat, lon
- integrated wave fields (no extra dimensions)
- 2D wave spectra        (extra dimensions: frequency,direction)
    - ERA5 provides 2D wave spectra with 24 directions and 30 frequencies

import relevant modules

In [1]:
import numpy as np
import netCDF4 as nc
from datetime import datetime
import pandas as pd
import xarray as xr
from scipy import integrate
import math

settings

In [2]:
ddir   = "/perm/pajk/marsData/seasonal/" # where data is stored

longitude=1      # demonstrative lon and lat coordinates (i.e. 1 deg N, 1 deg E)
latitude=1       # demonstrative lon and lat coordinates (i.e. 1 deg N, 1 deg E)

read in and decode spectra in GRIB
- as per https://github.com/nguyenquangchien/OceanSpec/blob/main/ECMWF_guide_ERA5.md
- summary of what is done and why, also from [above page](https://github.com/nguyenquangchien/OceanSpec/blob/main/ECMWF_guide_ERA5.md)
  - assign directional coordinates in degrees and convert to radians 
      - The direction bins start at 7.5 degree and increase by 15 degrees until 352.5, with 90 degree being towards the east (Oceanographic convention)
  - assign frequency coordinates
    - The frequency bins are non-linearly spaced. The first bin is $f=0.03453$ Hz and the following bins are: $f(n) = 1.1 f(n-1); n=2,30 $ 
  - decode data
    - The data provided is the $log_{10}$ of spectra density. To obtain the spectral density one has to take $spectra=10 ^ {data}$. This will give the units 2D wave spectra as $m^{2} \ s \ radian^{-1} $. 

In [3]:
dfSpec = xr.open_dataset(ddir+"era5_wave_20190601_20190601.spectra.grib",engine='cfgrib') # read in data
dfSpec = dfSpec.assign_coords(direction=np.deg2rad(np.arange(7.5, 352.5 + 15, 15)))       # assign directional coordinates in radians (see deg2rad function)
dfSpec = dfSpec.assign_coords(frequency=np.full(30, 0.03453) * (1.1 ** np.arange(0, 30))) # assign frequency coordinates
dfSpec = 10 ** dfSpec
dfSpec = dfSpec.fillna(0)

# reduce data size for convenience of calculations
dfSpec = dfSpec.sel(latitude=latitude,longitude=longitude)                                # select for lon and lat coordinates
# dfSpec = dfSpec.isel(time=0)                                                            # optionally select for only one time

# relabel for convenience
directions=dfSpec.direction.values
frequencies=dfSpec.frequency.values

print check on directions and frequencies that we have created

In [4]:
print('directions (in radians): \n',directions ,'\n')
print('frequencies: \n',frequencies)

directions (in radians): 
 [0.13089969 0.39269908 0.65449847 0.91629786 1.17809725 1.43989663
 1.70169602 1.96349541 2.2252948  2.48709418 2.74889357 3.01069296
 3.27249235 3.53429174 3.79609112 4.05789051 4.3196899  4.58148929
 4.84328867 5.10508806 5.36688745 5.62868684 5.89048623 6.15228561] 

frequencies: 
 [0.03453    0.037983   0.0417813  0.04595943 0.05055537 0.05561091
 0.061172   0.0672892  0.07401812 0.08141993 0.08956193 0.09851812
 0.10836993 0.11920693 0.13112762 0.14424038 0.15866442 0.17453086
 0.19198394 0.21118234 0.23230057 0.25553063 0.28108369 0.30919206
 0.34011127 0.3741224  0.41153464 0.4526881  0.49795691 0.5477526 ]


read in integrated wave fields computed online in the wave model
- significant wave height
- mean wave period
- mean wave direction

In [5]:
dfOnln = xr.open_dataset(ddir+"era5_wave_20190601_20190601.hs.grib"      ,engine='cfgrib')   # read in data
# dfOnln = dfOnln.isel(time=0)                                                               # optionally select for only one time
dfOnln = dfOnln.sel(latitude=latitude,longitude=longitude)                                   # select for lon and lat coordinates

simple print check of the values contained in the integrated wave fields

In [16]:
print('\n Significant wave height')
print('HS =',dfOnln.swh.values)  
print('\n Mean wave period (first moment)')
print('MWP =',dfOnln.mp1.values)
print('\n Mean wave direction')
print('MWD =',dfOnln.mwd.values)


 Significant wave height
HS = [1.6976986 1.6730528 1.6487865 1.6266775 1.6090908 1.5956926 1.584404
 1.5872974 1.5741901 1.5603943 1.5425949 1.5271301 1.5171185 1.5014725
 1.4866343 1.4729338 1.4594841 1.4469767 1.4361181 1.4277983 1.4203119
 1.4119978 1.4033794 1.3941317]

 Mean wave period (first moment)
MWP = [6.3466425 6.346319  6.395624  6.4604063 6.503525  6.517645  6.522339
 6.5561237 6.5943785 6.641906  6.7564297 6.8245926 6.867902  6.876938
 6.878601  6.866356  6.857791  6.8391867 6.7997293 6.731063  6.66319
 6.6180487 6.589011  6.5654173]

 Mean wave direction
MWD = [184.54587 184.40384 184.3642  184.39423 184.40012 184.30847 184.13159
 183.94653 183.8454  183.76694 183.77129 183.75757 183.71483 183.64932
 183.6015  183.56256 183.54709 183.5386  183.50047 183.38774 183.23769
 183.08165 182.97423 182.86333]


Integrations to calculate the integrated wave parameters from the spectrum

- follows Bidlot (2020) 


References
- Bidlot J., Ocean wave model output parameters. 2020, URL https://confluence.ecmwf.int/display/CKB/ECMWF+Model+Documentation?preview=/59774192/59774191/wave_parameters.pdf.



In [19]:
M0 = integrate.trapezoid(dfSpec['d2fd'].values                   , x=frequencies)
M0 = integrate.trapezoid(M0                                      , x=directions)

M1 = integrate.trapezoid(dfSpec['d2fd'].values *  frequencies    , x=frequencies)
M1 = integrate.trapezoid(M1                                      , x=directions)

M2 = integrate.trapezoid(dfSpec['d2fd'].values * (frequencies**2), x=frequencies)
M2 = integrate.trapezoid(M2                                      , x=directions)

SF = integrate.trapezoid(dfSpec['d2fd'].values                   , x=frequencies)
SF = integrate.trapezoid(SF*np.sin(directions)                   , x=directions)

CF = integrate.trapezoid(dfSpec['d2fd'].values                   , x=frequencies)
CF = integrate.trapezoid(CF*np.cos(directions)                   , x=directions)


try:   HS  = 4.04 * math.sqrt(M0)
except:HS  = 4.04 * M0**0.5

TM1 =           M0 / M1
try:   TM2 = math.sqrt(M0 / M2)
except:TM2 = (M0 / M2)**0.5
try:   MWD=np.degrees(np.arctan2(SF,CF))
except:MWD=np.degrees(np.arctan(SF,CF))

print check of the integrated values computed from the spectrum

In [20]:
print('\n Significant wave height')
print('HS =',HS)
print('\n Mean wave period (first moment)')
print('TM1=',TM1)
print('\n Mean wave direction')
print('MWD=',MWD)


 Significant wave height
HS = [1.56315652 1.53806303 1.51442198 1.49272466 1.47678181 1.46488042
 1.45417787 1.45624735 1.44581069 1.43137364 1.41450121 1.4017667
 1.39419099 1.37929021 1.3648753  1.351614   1.33913858 1.32752646
 1.31736701 1.3093493  1.30311414 1.295313   1.28837151 1.27990537]

 Mean wave period (first moment)
TM1= [6.4833933  6.48085768 6.51963118 6.57087719 6.60985681 6.62850533
 6.63388018 6.66774205 6.71233845 6.75328523 6.84372116 6.91588831
 6.96818448 6.9872519  6.99526898 6.98807801 6.98968605 6.9792659
 6.95707065 6.90772319 6.84909135 6.804005   6.77150094 6.74546492]

 Mean wave direction
MWD= [5.69001974 5.50864948 5.48253841 5.50715436 5.5243356  5.45375745
 5.1799495  4.9222762  4.85512145 4.70181728 4.77314561 4.75143216
 4.70338613 4.65900654 4.61781479 4.52417649 4.63316992 4.49849147
 4.5353985  4.35060035 4.19771626 4.01828743 3.97232868 3.77277146]


Differences explained
- Significant wave height and mean wave period differ by about 5-10%, reasons for this are
  1.	As explained in chapter 10 of https://www.ecmwf.int/en/elibrary/81373-ifs-documentation-cy48r1-part-vii-ecmwf-wave-model
  All integral parameters (except the mean square slope)  are computed over the full frequency range ( 0 to infinity) during the model run by summing over the discretised frequency bins and adding the analytical contribution for frequency above the last discretised frequency by assuming that above that frequency the spectrum is represented by a $f^{-5}$ tail
  This high frequency contribution, especially for the significant wave height is usually relatively small.
  2.	When encoding the 2d spectra, small spectral components below $10^{-3} – 10^{-4} \ m^2 \ radian^{-1}$ are discarded and encoded as missing.
  The other components are then encoded as the $log_{10}$ of the values using only 9 bits, whereas the integral parameters are encoded with 16 bits precission.
  Again, this procedure will result in relatively small differences.
  3.	The data for ERA5, are generally disseminated to users on a regular lat-lon grid (0.5 x 0.5) instead of the original native grid.
  The interpolation procedure is different for different wave parameters:
  Significant wave height is interpolated to the target grid using weighted average from the 3 closest native grid points,
  Whereas spectra are not interpolated and the value from the native grid closest to the target point on the lat-lon grid is used.
  
- Mean wave direction differ by $180 {\degree}$, reasons for this are
  - directional fields retrieved as a 2d field follows the atmospheric convention, but that the spectra follows the oceanic convention. These two conventions are $180 {\degree}$ separated.